# Models
In this notebook, we'll start to explore what models we can use to predict the output of solar panels based on the weather data. We'll start with simple models such as linear regression and nearest neighbors, decision trees and then move on to more complex ones such as random forests, gradient boosting and SVMs.

## Imports

In [1]:
# Loop printing
from tqdm import tqdm

# Data management
import pandas as pd

# Test and train split and mean squared error metric
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Cross validation grid search
from sklearn.model_selection import GridSearchCV

# Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

## Data Loading

In [2]:
df = pd.read_csv('../data/cleaned.csv', sep=';', index_col=1)
df = df.rename(columns={'Unnamed: 0': 'Timestamp'})
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df['Hour'] = df['Timestamp'].dt.hour + df['Timestamp'].dt.minute / 60

df = df.drop(df.columns[9:19], axis=1)
df = df.drop(columns=['Timestamp'])
df

,Power_1,Power_2,Power_3,Power_4,Power_5,Power_6,Power_7,Power_8,SWD,SWDtop,...,TT2M,ST,RH2m,WS10m,WS100m,PREC,SNOW,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
577035.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.38,8.38,93.34,3.46,9.10,0.0,0.0,1,1,2.25
577038.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.52,8.52,92.18,3.81,9.52,0.0,0.0,1,1,2.50
577041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.66,8.66,90.91,4.15,9.66,0.0,0.0,1,1,2.75
577044.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.76,8.76,90.18,4.36,9.50,0.0,0.0,1,1,3.00
577047.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.74,8.74,90.39,4.38,9.16,0.0,0.0,1,1,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.14,13.14,72.48,7.39,13.79,0.0,0.0,12,31,21.50
667204.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.13,13.13,72.07,7.46,13.93,0.0,0.0,12,31,21.75
667207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.12,13.12,71.67,7.52,14.04,0.0,0.0,12,31,22.00


In [3]:
target_cols = ['Power_1', 'Power_2', 'Power_3', 'Power_4', 'Power_5', 'Power_6', 'Power_7', 'Power_8']
target = df[target_cols]
target

,Power_1,Power_2,Power_3,Power_4,Power_5,Power_6,Power_7,Power_8
id,,,,,,,,
577035.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577038.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577044.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577047.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
667201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667204.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
features = df.drop(columns=target_cols, axis=1)
features

,SWD,SWDtop,CU,CM,CD,TT2M,ST,RH2m,WS10m,WS100m,PREC,SNOW,Month,Day,Hour
id,,,,,,,,,,,,,,,
577035.0,0.0,0.0,0.00,0.0,0.0,8.38,8.38,93.34,3.46,9.10,0.0,0.0,1,1,2.25
577038.0,0.0,0.0,0.00,0.0,0.0,8.52,8.52,92.18,3.81,9.52,0.0,0.0,1,1,2.50
577041.0,0.0,0.0,0.00,0.0,0.0,8.66,8.66,90.91,4.15,9.66,0.0,0.0,1,1,2.75
577044.0,0.0,0.0,0.00,0.0,0.0,8.76,8.76,90.18,4.36,9.50,0.0,0.0,1,1,3.00
577047.0,0.0,0.0,0.00,0.0,0.0,8.74,8.74,90.39,4.38,9.16,0.0,0.0,1,1,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667201.0,0.0,0.0,0.76,0.0,0.0,13.14,13.14,72.48,7.39,13.79,0.0,0.0,12,31,21.50
667204.0,0.0,0.0,1.00,0.0,0.0,13.13,13.13,72.07,7.46,13.93,0.0,0.0,12,31,21.75
667207.0,0.0,0.0,1.00,0.0,0.0,13.12,13.12,71.67,7.52,14.04,0.0,0.0,12,31,22.00


## Splitting into Train, Validation and Test Sets

In [5]:
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.25, random_state=42)
features_val, features_test, target_val, target_test = train_test_split(features_temp, target_temp, test_size=0.5, random_state=42)

print('Training features shape:', features_train.shape)
print('Validation features shape:', features_val.shape)
print('Testing features shape:', features_test.shape)
print('Training target shape:', target_train.shape)
print('Validation target shape:', target_val.shape)
print('Testing target shape:', target_test.shape)

Training features shape: (21637, 15)
Validation features shape: (3606, 15)
Testing features shape: (3607, 15)
Training target shape: (21637, 8)
Validation target shape: (3606, 8)
Testing target shape: (3607, 8)


## Models Development

### Linear Regression

In [6]:
lr = []
predictions = []
mse = []

for i in tqdm(range(1, 9)):
    lr.append(LinearRegression().fit(features_train, target_train['Power_' + str(i)]))
    predictions.append(lr[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

lr_result = pd.DataFrame({'Model': 'Linear Regression', 'Power': range(1, 9), 'MSE': mse, 'Params': 'None'})
lr_result

100%|██████████| 8/8 [00:00<00:00, 68.72it/s]


,Model,Power,MSE,Params
0,Linear Regression,1,7.309333e+07,None
1,Linear Regression,2,7.689520e+07,None
2,Linear Regression,3,6.782160e+07,None
3,Linear Regression,4,6.254578e+07,None
4,Linear Regression,5,1.316790e+07,None
5,Linear Regression,6,1.332025e+07,None
6,Linear Regression,7,1.367990e+07,None
7,Linear Regression,8,3.142337e+08,None


### Nearest Neighbors

In [7]:
knn = []
knn_params = []
predictions = []
mse = []

params = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49]}

for i in tqdm(range(1, 9)):
    knn.append(GridSearchCV(KNeighborsRegressor(), param_grid=params, cv=5).fit(features_train, target_train['Power_' + str(i)]))
    knn_params.append(knn[i-1].best_params_)
    predictions.append(knn[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

knn_result = pd.DataFrame({'Model': 'KNN', 'Power': range(1, 9), 'MSE': mse, 'Params': knn_params})
knn_result

100%|██████████| 8/8 [03:00<00:00, 22.52s/it]


,Model,Power,MSE,Params
0,KNN,1,6.564239e+07,{'n_neighbors': 23}
1,KNN,2,6.944180e+07,{'n_neighbors': 23}
2,KNN,3,6.094527e+07,{'n_neighbors': 25}
3,KNN,4,5.604094e+07,{'n_neighbors': 23}
4,KNN,5,1.190184e+07,{'n_neighbors': 23}
5,KNN,6,1.201344e+07,{'n_neighbors': 23}
6,KNN,7,1.232869e+07,{'n_neighbors': 23}
7,KNN,8,3.181898e+08,{'n_neighbors': 49}


### Decision Trees

In [8]:
dt = []
dt_params = []
predictions = []
mse = []

param_grid = {
    'max_depth': [None, 10, 20, 30], # Maximum depth of the tree
    'min_samples_split': [2, 5, 10], # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],   # Minimum number of samples required at each leaf node
}

for i in tqdm(range(1, 9)):
    dt.append(GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid, cv=5).fit(features_train, target_train['Power_' + str(i)]))
    dt_params.append(dt[i-1].get_params())
    predictions.append(dt[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

dt_result = pd.DataFrame({'Model': 'Decision Tree', 'Power': range(1, 9), 'MSE': mse, 'Params': dt_params})
dt_result

100%|██████████| 8/8 [03:35<00:00, 27.00s/it]


,Model,Power,MSE,Params
0,Decision Tree,1,6.167236e+07,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
1,Decision Tree,2,6.466225e+07,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
2,Decision Tree,3,5.599870e+07,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
3,Decision Tree,4,5.152444e+07,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
4,Decision Tree,5,8.614799e+06,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
5,Decision Tree,6,8.802316e+06,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
6,Decision Tree,7,8.557739e+06,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."
7,Decision Tree,8,3.437403e+08,"{'cv': 5, 'error_score': nan, 'estimator__ccp_..."


### Random Forests

In [9]:
rf = []
rf_params = []
predictions = []
mse = []

for i in tqdm(range(1, 9)):
    rf.append(RandomForestRegressor(random_state=42).fit(features_train, target_train['Power_' + str(i)]))
    rf_params.append(rf[i-1].get_params())
    predictions.append(rf[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

rf_result = pd.DataFrame({'Model': 'Random Forest', 'Power': range(1, 9), 'MSE': mse, 'Params': rf_params})
rf_result

100%|██████████| 8/8 [01:41<00:00, 12.67s/it]


,Model,Power,MSE,Params
0,Random Forest,1,3.575971e+07,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
1,Random Forest,2,3.673638e+07,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,Random Forest,3,3.458360e+07,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
3,Random Forest,4,3.156719e+07,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
4,Random Forest,5,5.211455e+06,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
5,Random Forest,6,5.231553e+06,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
6,Random Forest,7,5.178944e+06,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,Random Forest,8,3.379356e+08,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."


### Gradient Boosting

In [10]:
gb = []
gb_params = []
predictions = []
mse = []

for i in tqdm(range(1, 9)):
    gb.append(GradientBoostingRegressor().fit(features_train, target_train['Power_' + str(i)]))
    gb_params.append(gb[i-1].get_params)
    predictions.append(gb[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

gb_result = pd.DataFrame({'Model': 'Gradient Boosting', 'Power': range(1, 9), 'MSE': mse, 'Params': gb_params})
gb_result

100%|██████████| 8/8 [00:32<00:00,  4.08s/it]


,Model,Power,MSE,Params
0,Gradient Boosting,1,5.155582e+07,<bound method BaseEstimator.get_params of Grad...
1,Gradient Boosting,2,5.548484e+07,<bound method BaseEstimator.get_params of Grad...
2,Gradient Boosting,3,4.861806e+07,<bound method BaseEstimator.get_params of Grad...
3,Gradient Boosting,4,4.449063e+07,<bound method BaseEstimator.get_params of Grad...
4,Gradient Boosting,5,8.948096e+06,<bound method BaseEstimator.get_params of Grad...
5,Gradient Boosting,6,9.041375e+06,<bound method BaseEstimator.get_params of Grad...
6,Gradient Boosting,7,8.964832e+06,<bound method BaseEstimator.get_params of Grad...
7,Gradient Boosting,8,3.617578e+08,<bound method BaseEstimator.get_params of Grad...


### SVMs

In [11]:
svm = []
svm_params = []
predictions = []
mse = []

for i in tqdm(range(1, 9)):
    svm.append(SVR().fit(features_train, target_train['Power_' + str(i)]))
    svm_params.append(svm[i-1].get_params)
    predictions.append(svm[i-1].predict(features_val))
    mse.append(mean_squared_error(target_val['Power_' + str(i)], predictions[i-1]))

svm_result = pd.DataFrame({'Model': 'Support Vector Machine', 'Power': range(1, 9), 'MSE': mse, 'Params': svm_params})
svm_result

100%|██████████| 8/8 [05:05<00:00, 38.13s/it]


,Model,Power,MSE,Params
0,Support Vector Machine,1,3.174133e+08,<bound method BaseEstimator.get_params of SVR()>
1,Support Vector Machine,2,3.192553e+08,<bound method BaseEstimator.get_params of SVR()>
2,Support Vector Machine,3,3.055526e+08,<bound method BaseEstimator.get_params of SVR()>
3,Support Vector Machine,4,2.703837e+08,<bound method BaseEstimator.get_params of SVR()>
4,Support Vector Machine,5,3.100377e+07,<bound method BaseEstimator.get_params of SVR()>
5,Support Vector Machine,6,3.145750e+07,<bound method BaseEstimator.get_params of SVR()>
6,Support Vector Machine,7,3.156851e+07,<bound method BaseEstimator.get_params of SVR()>
7,Support Vector Machine,8,3.330551e+08,<bound method BaseEstimator.get_params of SVR()>


### Choosing best models

In [13]:
best_models = pd.DataFrame({'Power': range(1,9)})
best_models['Model'] = None
best_models['MSE'] = None
best_models['Params'] = None

models = [lr_result, knn_result, dt_result, rf_result, gb_result, svm_result]
for i in range(1,9):
    best_model = min(models, key=lambda x: x['MSE'][i-1])
    best_models['Model'][i-1] = best_model['Model'][0]
    best_models['MSE'][i-1] = best_model['MSE'][i-1]
    best_models['Params'][i-1] = best_model['Params'][i-1]

best_models = best_models.set_index('Power')

best_models

/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/ipykernel_4805/2571922508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_models['Model'][i-1] = best_model['Model'][0]
/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/ipykernel_4805/2571922508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_models['MSE'][i-1] = best_model['MSE'][i-1]
/var/folders/d5/xjnfc_z96714df80fs1j8sg80000gn/T/ipykernel_4805/2571922508.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,Model,MSE,Params
Power,,,
1,Random Forest,35759710.118214,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,Random Forest,36736377.638679,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
3,Random Forest,34583595.374903,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
4,Random Forest,31567191.697783,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
5,Random Forest,5211455.240489,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
6,Random Forest,5231553.322003,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,Random Forest,5178943.932301,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
8,Linear Regression,314233663.547836,None
